In [70]:
### Step 1. Build aindex over the reference genome

ref_file = "/media/eternus1/nfs/projects/thiomargarita/reference/2926625205.fna"
output_prefix = "/media/eternus1/nfs/projects/thiomargarita/reference/2926625205"
command = f"python ~/Dropbox/workspace/PyBioSnippets/hiseq/compute_aindex.py -i {ref_file} -o {output_prefix} -t fasta"
command


'python ~/Dropbox/workspace/PyBioSnippets/hiseq/compute_aindex.py -i /media/eternus1/nfs/projects/thiomargarita/reference/2926625205.fna -o /media/eternus1/nfs/projects/thiomargarita/reference/2926625205 -t fasta'

In [73]:
### Step 2. Read all pairs from the raw data folder, we use trimmed data

import PyExp

data_folder = "/media/eternus1/nfs/projects/thiomargarita/samples_data/"

all_files = {}
for file_name in PyExp.sc_iter_filepath_folder(data_folder):
    if not file_name.endswith("fastq"):
        continue
    if not "trim" in file_name:
        continue
    name = file_name.split("/")[-2]
    if "trim_1" in file_name:
        all_files.setdefault(name, [None, None])
        all_files[name][0] = file_name
        continue
    if "trim_2" in file_name:
        all_files.setdefault(name, [None, None])
        all_files[name][1] = file_name
        continue

In [75]:
### Step 3. Create a list of task to compute

import os

command = "time ~/Dropbox/workspace/PyBioSnippets/Pantheon/bin/V5_ref_cov_stats.exe %s %s %s 150 /media/eternus1/nfs/projects/thiomargarita/reference/2926625205.23.pf /media/eternus1/nfs/projects/thiomargarita/reference/2926625205.23"
output_pattern = "/media/eternus1/nfs/projects/users/akomissarov/thiomag/stats/%s.covs"
compute_list_file = "/media/eternus1/nfs/projects/users/akomissarov/thiomag/compute_list.list"
name2stat_file = {}
done = 0
total = 0

do_gzip = False

with open(compute_list_file, "w") as fw:

    for name in all_files:
        f1, f2 = all_files[name]
        if f1 is None:
            f1 = f2
        if f2 is None:
            f2 = f1
        prev = name.split(".fastq")[0]
        output_file = output_pattern % name
        if os.path.isfile(output_file+".cov.stat"):
            done += 1
            name2stat_file[name] = output_file+".cov.stat"
            continue
        total += 1
        cc = ""
        if f1.endswith("gz"):
            cc += "gzip -d %s; " % f1
            f1 = f1[:-3]
        if f2.endswith("gz"):
            cc += "gzip -d %s; " % f2
            f2 = f2[:-3]
        cc += command % (f1, f2, output_file)
        if do_gzip:
            cc += "; gzip -9 %s; " % f1
            cc += "gzip -9 %s " % f2
        cc = cc.strip()
        fw.write(f"{cc}\n")
    
print(f"Computed: {done}, to compute: {total}")

Computed: 80, to compute: 0


In [76]:
### Step 4. execute them

command = """less /media/eternus1/nfs/projects/users/akomissarov/thiomag/compute_list.list | xargs -I{} -P 20 sh -c '{}'"""
command

"less /media/eternus1/nfs/projects/users/akomissarov/thiomag/compute_list.list | xargs -I{} -P 20 sh -c '{}'"

In [67]:
### Step 5. Parse data

name2stats = {}
for ii, name in enumerate(name2stat_file):
    print(ii, end=" ")
    with open(name2stat_file[name]) as fh:
        total = 0
        zeros = 0
        ones = 0
        good_10 = 0
        good_20 = 0
        if name in name2stats:
            continue
        for line in fh:
            d = list(map(int, line.strip().split("\t")))
            total += 1
            if d[3] == 0 and d[6] == 0:
                zeros += 1
                continue
            if d[3] + d[6] > 0:
                ones += 1
            if d[3] + d[6] >= 10:
                good_10 += 1
            if d[3] + d[6] >= 20:
                good_20 += 1
        name2stats[name] = (total, zeros, ones, good_10, good_20)

names = list(name2stats.keys())
names.sort()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 

In [85]:
!ls -hlrt /media/eternus1/nfs/projects/thiomargarita/reference/2926625205.fna

-rwxrwxrwx 1 2004 1901 17M Jul 15  2022 /media/eternus1/nfs/projects/thiomargarita/reference/2926625205.fna


In [81]:
### Step 6. Create summary table, 
### after then it can be copied to any editor where you could replace \s with \t 
### and then you could copy paste to google spredsheet and then to google doc

expected_genome_size = 17000000
reads_length = 150
k = 23
number_of_kmers_per_pair = 2 * (reads_length - k + 1)
number_of_reads_for_x1_coverage = expected_genome_size / number_of_kmers_per_pair


In [84]:
for name in names:
    total, zeros, ones, good_10, good_20 = name2stats[name]
    print(name, end="\t")
    print(f"{round(100.*zeros/total, 2)}%", end="\t")
    print(f"{total-zeros}bp", end="\t")
    print(f"x{round((total-zeros)/number_of_reads_for_x1_coverage,2)}")

SRR18713657	99.77%	3513bp	x0.05
SRR18713668	99.11%	32723bp	x0.49
SRR18713683	97.06%	45324bp	x0.68
SRR18713684	99.76%	3345bp	x0.05
SRR18713715	99.69%	5376bp	x0.08
SRR18713716	99.99%	214bp	x0.0
SRR18713744	99.68%	6274bp	x0.09
SRR18713805	99.93%	2511bp	x0.04
SRR18713845	99.81%	4101bp	x0.06
SRR18714148	97.46%	67966bp	x1.02
SRR18714150	99.82%	3366bp	x0.05
SRR18714151	98.76%	33984bp	x0.51
SRR18714292	98.01%	28647bp	x0.43
SRR18714293	99.98%	541bp	x0.01
SRR18714294	99.97%	1440bp	x0.02
SRR18714295	93.3%	221952bp	x3.34
SRR18714297	99.93%	1477bp	x0.02
SRR18714343	99.99%	377bp	x0.01
SRR18714529	99.96%	870bp	x0.01
SRR18715514	99.22%	12599bp	x0.19
SRR18715586	99.51%	9071bp	x0.14
SRR18715604	99.94%	812bp	x0.01
SRR18715623	99.69%	8086bp	x0.12
SRR18715702	99.66%	12826bp	x0.19
SRR18715703	98.72%	28076bp	x0.42
SRR18715705	79.17%	536803bp	x8.08
SRR18715710	99.99%	383bp	x0.01
SRR18715723	99.99%	432bp	x0.01
SRR18715745	99.59%	11681bp	x0.18
SRR18715959	99.33%	11462bp	x0.17
SRR18715961	99.94%	502bp	x0.01
SRR1

In [69]:
### Step 7. Seperate target and contamination

output_target_folder = "/media/eternus1/nfs/projects/users/akomissarov/thiomag/target/"
output_contamin_folder = "/media/eternus1/nfs/projects/users/akomissarov/thiomag/contamin/"
pos_file = os.path.join(output_target_folder, "%s.target_%s.fastq")
neg_file = os.path.join(output_contamin_folder, "%s.contam_%s.fastq")

def read_fastq(fh):
    d = {
        "header": fh.readline(),
        "seq": fh.readline(),
        "strand": fh.readline(),
        "q": fh.readline(),
    }
    return d

def write_fastq(d, fw):
    fw.write(d["header"])
    fw.write(d["seq"])
    fw.write(d["strand"])
    fw.write(d["q"])

for ii, name in enumerate(all_files):
    print(ii, end=" ")
    f1, f2 = all_files[name]
    stats_file = name2stat_file[name]
    
    w1 = pos_file % (name, 1)
    
    if os.path.isfile(w1):
        continue
    fh_1 = open(f1)
    fh_2 = open(f2)    
    w2 = pos_file % (name, 2)
    n1 = neg_file % (name, 1)
    n2 = neg_file % (name, 2)
    
    fh_s = open(name2stat_file[name])
    wh_pos_1 = open(w1, "w")
    wh_pos_2 = open(w2, "w")
    wh_neg_1 = open(n1, "w")
    wh_neg_2 = open(n2, "w")
    while True:
        d = fh_s.readline().strip().split("\t")
        if not d or not d[0]:
            break
        stat_line = list(map(int, d))
        r1 = read_fastq(fh_1)
        r2 = read_fastq(fh_2)
        if stat_line[1]-stat_line[2] + stat_line[4]-stat_line[5] == 0:
            write_fastq(r1, wh_neg_1)
            write_fastq(r2, wh_neg_2)
        else:
            write_fastq(r1, wh_pos_1)
            write_fastq(r2, wh_pos_2)
        
    fh_1.close()
    fh_2.close()
    wh_pos_1.close()
    wh_pos_2.close()
    wh_neg_1.close()
    wh_neg_2.close()
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 